In [ ]:
# 1. Environment check
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)


In [2]:
# 2. Imports
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory

from sklearn.metrics import classification_report, confusion_matrix
import itertools


In [3]:
# 3. Dataset path & params
dataset_path = r"C:\Users\srina\Downloads\archive\garbage-dataset"   # update if needed
img_size = (224, 224)
batch_size = 32


In [ ]:
# 4. Load dataset (80% train, 20% val)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes found:", class_names)
num_classes = len(class_names)


In [5]:
# 5. Performance (optional but recommended)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
# 6. Build model
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
# 7. Train model
EPOCHS = 10   # change to 20/30 if you want
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


In [ ]:
# 8. Plot metrics
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training & Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training & Validation Loss')
plt.show()


In [ ]:
# 9A. Test single image using in-memory model (no reload needed)
img_path = r"C:\Users\srina\Downloads\archive\garbage-dataset\plastic\plastic_93.jpg"
img = image.load_img(img_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

prediction = model.predict(img_array)     # using the model in memory
predicted_class = class_names[np.argmax(prediction)]
confidence = round(np.max(prediction) * 100, 2)
print("Predicted Class:", predicted_class)
print("Confidence:", confidence, "%")


In [ ]:
# 9B. If you restarted the kernel or opened new session: load model first
model_loaded = keras.models.load_model("waste_classifier_model.keras")
print("Loaded model:", model_loaded)

img_path = r"C:\Users\srina\Downloads\archive\garbage-dataset\plastic\plastic_93.jpg"
img = image.load_img(img_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

prediction = model_loaded.predict(img_array)
predicted_class = class_names[np.argmax(prediction)]
confidence = round(np.max(prediction) * 100, 2)
print("Predicted Class:", predicted_class)
print("Confidence:", confidence, "%")


In [ ]:
# 10. Save model (saves a separate file in working directory)
model.save("waste_classifier_model.keras")
print("Model saved to waste_classifier_model.keras")
